<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from __future__ import print_function

import sys
import requests
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random

In [2]:
text = open('plays.txt', 'r').read()

In [3]:
chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

len(chars)

65

In [4]:
encoded = [char_int[c] for c in text]
len(encoded)

1115394

In [5]:
maxlen = 64
step = 16

sequences = []
next_chars = []

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 69709


In [6]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

x.shape, y.shape

((69709, 64, 65), (69709, 65))

In [7]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

W0127 23:57:08.525154  8588 deprecation.py:506] From C:\Users\John\Anaconda3\envs\U4-S2-NN\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
def on_epoch_end(epoch, _):
        
    if epoch % 16 == 0:
    
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)

        generated = ''

        sentence = text[start_index: start_index + maxlen]
        generated += sentence

        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(128):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature=1.0)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [10]:
model.fit(x, y,
          batch_size=128,
          epochs=128,
          callbacks=[print_callback])

W0127 23:57:08.845052  8588 deprecation.py:323] From C:\Users\John\Anaconda3\envs\U4-S2-NN\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/128
69632/69709 [============================>.] - ETA: 0s - loss: 3.0081
----- Generating text after Epoch: 0
----- Generating with seed: "uch liberty, my Lucio, liberty:
As surfeit is the father of much"
uch liberty, my Lucio, liberty:
As surfeit is the father of muche aslcle he ges sertis thsrmoad besite gna thl hos the ap chelt.

 pRDian:

Chemedc eoles mios .

ShaisEwhipuer, uinh NhC atd ci
69709/69709 [==============================] - 42s 600us/sample - loss: 3.0077
Epoch 2/128
69709/69709 [==============================] - 40s 580us/sample - loss: 2.4234
Epoch 3/128
69709/69709 [==============================] - 44s 628us/sample - loss: 2.2726
Epoch 4/128
69709/69709 [==============================] - 44s 636us/sample - loss: 2.1809
Epoch 5/128
69709/69709 [==============================] - 44s 637us/sample - loss: 2.1093
Epoch 6/128
69709/69709 [==============================] - 44s 635us/sample - loss: 2.0476
Epoch 7/128
69709/69709 [==============================] - 

In [11]:
start_index = random.randint(0, len(text) - maxlen - 1)

generated = ''

sentence = text[start_index: start_index + maxlen]
generated += sentence

print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

for i in range(2048):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_int[char]] = 1

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, temperature=1.0)
    next_char = int_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()

----- Generating with seed: "las! I have show'd too much
The rashness of a woman: he is touch"
las! I have show'd too much
The rashness of a woman: he is toucher and you:
That thou rampores-baved it so deaf vouter.

SICINIS:
Sir, been sir? I am fullinessor:
Let this assition.

Lord: re'lood he rodory.

MartHNS'S:

PAMIST:
Were is seapter me,
Why tadest one doy is gesare
In houses time in a many doung talk'd me?
When, then will queen and the boun, sir,'t;
Browerd good with a sable. Lord is his triafester,
ED foogow cotciatu.

NoClloon:
my tooad's bese.

ISCAMENS:
Sire sonce I sting a that exely that change to your?

Throwo ringranvermane:
He ramed, good me thy grach his blood towrad:
For what noth bait butnist grant, brother.

DUCHORIO:
I what's every hear neverest dishier to his poste,
Firen, they streatter trabous from fliar no
Wid, beith their are connus to the uss.

SAMPPCLIUTIO:
Hearsel, is your linely:
Lord: then dayour pranfon as exery dis conder
That I have thy drow clonngly you

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN